In [ ]:
import numpy as np
import gc

In [ ]:
a = []
b = [1,2,3]
c = [4,5,6]

a.append(b)
a.append(c)

del b
del c
gc.collect()



print(a)
print(b)
print(c)

In [ ]:
# create an empty 3D array with 2 layers, 3 rows and 4 columns
arr = np.empty((2, 3, 4))

# print the array
print(arr)

In [ ]:
#na = np.array([[1, 2, 3]], dtype=np.float64)
na = np.array()

#na = np.append(na, [[4, 5, 6]], axis=0)


na

# LSTM Issue

In [ ]:
!pip install tensorflow_datasets

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib

dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.padded_batch(BATCH_SIZE, train_dataset.output_shapes)

test_dataset = test_dataset.padded_batch(BATCH_SIZE, test_dataset.output_shapes)
encoder = info.features['text'].encoder


model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset, 
                    validation_steps=30)

## Coordinate Heatmap

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def create_heatmap(screen_coor_x, screen_coor_y):
    # Determine the size of the heatmap
    max_x = int(np.ceil(max(screen_coor_x)))
    max_y = int(np.ceil(max(screen_coor_y)))

    # Create a 2D histogram of the coordinates
    heatmap, xedges, yedges = np.histogram2d(screen_coor_y, screen_coor_x, bins=(max_y, max_x))

    # Create the heatmap image and display it
    plt.imshow(heatmap.T, origin='lower')
    plt.show()



# Example usage
session_1_screen_coor_x = [1.2, 2.4, 3.1, 4.5, 2.2]
session_1_screen_coor_y = [3.5, 1.7, 4.8, 2.1, 5.0]

create_heatmap(session_1_screen_coor_x, session_1_screen_coor_y)

In [ ]:
import pandas as pd

# load the source training set
df_source = pd.read_csv('data/train.csv.gz', compression='gzip', index_col=1)

print(df_source.shape)
with pd.option_context('display.max_columns', None):
    display(df_source.head(3))

In [ ]:
def get_session_screen_coor(data: df_source, session_id: int):
    # get the session data
    session_data = df_source[df_source['session_id'] == session_id]
    session_data = session_data[['screen_coor_x', 'screen_coor_y']]
    session_data = session_data.dropna()

    # get the screen coordinates
    screen_coor_x = session_data['screen_coor_x'].values
    screen_coor_y = session_data['screen_coor_y'].values
    
    return screen_coor_x, screen_coor_y

In [ ]:
session_1_screen_coor_x, session_1_screen_coor_y =  \
    get_session_screen_coor(df_source, 20090312431273200)

In [ ]:
session_2_screen_coor_x, session_2_screen_coor_y =  \
    get_session_screen_coor(df_source, 21040510125933256)

In [ ]:
#20090312433251036
#22100221145014656

# this one has a bigger max x value
session_3_screen_coor_x, session_3_screen_coor_y =  \
    get_session_screen_coor(df_source, 21040510125933256)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def create_heatmap(screen_coor_x, screen_coor_y):
    # Determine the size of the heatmap
    max_x = int(np.ceil(max(screen_coor_x)))
    max_y = int(np.ceil(max(screen_coor_y)))

    # Create a 2D histogram of the coordinates
    heatmap, xedges, yedges = np.histogram2d(screen_coor_y, screen_coor_x, bins=50)

    # Create the heatmap image and display it
    # plt.imshow(heatmap.T, origin='lower')
    # plt.show()


    extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
    plt.imshow(heatmap.T, cmap='hot', extent=extent, origin='lower')

# Example usage
#create_heatmap(screen_coor_x, screen_coor_y)

In [ ]:
def create_heatmap(screen_coor_x, screen_coor_y):
    """
    Make the points stand out more by applying a logarithmic transformation
    """
    # Create the 2D histogram
    heatmap, xedges, yedges = np.histogram2d(screen_coor_y, screen_coor_x, bins=50)

    # Apply logarithmic transformation
    heatmap = np.log(heatmap + 1)

    # Plot the heatmap
    extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
    plt.imshow(heatmap.T, cmap='hot', extent=extent, origin='lower')
    plt.axis('off')
    plt.show()

    return heatmap

#create_heatmap(screen_coor_x, screen_coor_y)

In [ ]:
session_1_heatmap = create_heatmap(session_1_screen_coor_x, session_1_screen_coor_y)
session_2_heatmap = create_heatmap(session_2_screen_coor_x, session_2_screen_coor_y)
session_3_heatmap = create_heatmap(session_3_screen_coor_x, session_3_screen_coor_y)

In [ ]:
print(session_1_heatmap.shape)
print(session_2_heatmap.shape)
print(session_3_heatmap.shape)

In [ ]:
from PIL import Image

def save_heatmap_to_image(heatmap):
    # Normalize the heatmap
    normalized_heatmap = (heatmap - np.min(heatmap)) / (np.max(heatmap) - np.min(heatmap))

    # Scale the heatmap to the range 0-255
    scaled_heatmap = (normalized_heatmap * 255).astype(np.uint8)

    # Create an image from the heatmap array
    img = Image.fromarray(scaled_heatmap)

    # Save the image to a file
    img.save('heatmap.png')

    return img

# test the function
simple_screen_coor_x = [1.2, 2.4, 3.1, 4.5, 2.2]
simple_screen_coor_y = [3.5, 1.7, 4.8, 2.1, 5.0]

#heatmap = create_heatmap(simple_screen_coor_x, simple_screen_coor_y)
heatmap = create_heatmap(session_1_screen_coor_x, session_1_screen_coor_y)
heatmap_image = save_heatmap_to_image(heatmap)
heatmap_array = np.array(heatmap_image)
heatmap_reshape = heatmap.reshape((heatmap.shape[0], heatmap.shape[1], 1))

#img = Image.fromarray(np.uint8(heatmap_reshape*255))
img = Image.fromarray(heatmap)
img = img.convert('RGB')
img.save('heatmap_reshape.png')

print(heatmap)

print(heatmap_array.shape)
print(heatmap_reshape.shape)

In [ ]:
max_x = np.max(session_1_screen_coor_x)
df_source[df_source['screen_coor_x'] > max_x]

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt

# def create_heatmap(screen_coor_x, screen_coor_y):
#     # Determine the size of the heatmap
#     max_x = int(np.ceil(max(screen_coor_x)))
#     max_y = int(np.ceil(max(screen_coor_y)))

#     # Create a 2D histogram of the coordinates
#     heatmap, xedges, yedges = np.histogram2d(screen_coor_y, screen_coor_x, bins=50)

#     # Create the heatmap image and display it
#     # plt.imshow(heatmap.T, origin='lower')
#     # plt.show()


#     extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
#     plt.imshow(heatmap.T, cmap='hot', extent=extent, origin='lower')

# # Example usage
# create_heatmap(screen_coor_x, screen_coor_y)

In [1]:
# Import TensorFlow and Keras
import tensorflow as tf
from tensorflow import keras

# Set the GPU memory from growth
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# Load the MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Preprocess the images by scaling them to the range [0, 1]
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture using a sequential API
model = keras.Sequential([
  keras.layers.Flatten(input_shape=(28, 28)), # Flatten the input images to a vector of 784 pixels
  keras.layers.Dense(128, activation='relu'), # Add a hidden layer with 128 neurons and ReLU activation
  keras.layers.Dense(10) # Add an output layer with 10 neurons for each class (0-9)
])

# Compile the model with an optimizer, a loss function and a metric
model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train the model on the training data for 10 epochs
model.fit(train_images, train_labels, epochs=10)

# Evaluate the model on the test data and print the results
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print('\nTest loss:', test_loss)
print('Test accuracy:', test_acc)

2023-03-03 18:50:46.345489: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-03-03 18:50:47.465206: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-03 18:50:47.466765: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-03 18:50:47.466879: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


11490434/11490434 [==============================] - 2s 0us/step


2023-03-03 18:50:49.879026: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-03 18:50:49.879495: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-03 18:50:49.879630: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-03 18:50:49.879734: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

Epoch 1/10


2023-03-03 18:50:51.113966: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-03-03 18:50:51.235922: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f953bfc8e80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-03-03 18:50:51.235941: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3080 Ti, Compute Capability 8.6
2023-03-03 18:50:51.238423: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-03-03 18:50:51.311060: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1875/1875 [==============================] - 3s 867us/step - loss: 0.2591 - accuracy: 0.9257
Epoch 2/10
1875/1875 [==============================] - 2s 890us/step - loss: 0.1172 - accuracy: 0.9653
Epoch 3/10
1875/1875 [==============================] - 2s 880us/step - loss: 0.0800 - accuracy: 0.9757
Epoch 4/10
1875/1875 [==============================] - 2s 873us/step - loss: 0.0613 - accuracy: 0.9815
Epoch 5/10
1875/1875 [==============================] - 2s 874us/step - loss: 0.0470 - accuracy: 0.9855
Epoch 6/10
1875/1875 [==============================] - 2s 872us/step - loss: 0.0370 - accuracy: 0.9883
Epoch 7/10
1875/1875 [==============================] - 2s 870us/step - loss: 0.0294 - accuracy: 0.9908
Epoch 8/10
1875/1875 [==============================] - 2s 876us/step - loss: 0.0243 - accuracy: 0.9926
Epoch 9/10
1875/1875 [==============================] - 2s 877us/step - loss: 0.0205 - accuracy: 0.9940
Epoch 10/10
1875/1875 [==============================] - 2s 870us/step - lo